In [1]:
import pandas as pd
import numpy as np

In [2]:
unspsc = pd.read_excel('tags_un.xlsx')

In [3]:
segment_to_sector = {
    "Apparel and Luggage and Personal Care Products": "Consumer Discretionary",
    "Building and Construction Machinery and Accessories": "Industrials",
    "Building and Facility Construction and Maintenance Services": "Industrials",
    "Chemicals including Bio Chemicals and Gas Materials": "Materials",
    "Cleaning Equipment and Supplies": "Industrials",
    "Distribution and Conditioning Systems and Equipment and Components": "Industrials",
    "Domestic Appliances and Supplies and Consumer Electronic Products": "Consumer Discretionary",
    "Editorial and Design and Graphic and Fine Art Services": "Communication Services",
    "Education and Training Services": "Consumer Discretionary",
    "Electrical Systems and Lighting and Components and Accessories and Supplies": "Industrials",
    "Electronic Components and Supplies": "Information Technology",
    "Engineering and Research and Technology Based Services": "Industrials",
    "Environmental Services": "Industrials",
    "Farming and Fishing and Forestry and Wildlife Contracting Services": "Industrials",
    "Farming and Fishing and Forestry and Wildlife Machinery and Accessories": "Industrials",
    "Financial and Insurance Services": "Financials",
    "Financial Instruments, Products, Contracts and Agreements": "Financials",
    "Food and Beverage Products": "Consumer Staples",
    "Fuels and Fuel Additives and Lubricants and Anti corrosive Materials": "Energy",
    "Furniture and Furnishings": "Consumer Discretionary",
    "Healthcare Services": "Health Care",
    "Humanitarian Relief Items, Kits, or Accessories": "Consumer Staples",
    "Industrial Cleaning Services": "Industrials",
    "Industrial Manufacturing and Processing Machinery and Accessories": "Industrials",
    "Industrial Production and Manufacturing Services": "Industrials",
    "Information Technology Broadcasting and Telecommunications": "Communication Services",
    "Laboratory and Measuring and Observing and Testing Equipment": "Health Care",
    "Land and Buildings and Structures and Thoroughfares": "Real Estate",
    "Law Enforcement and National Security and Security and Safety Equipment and Supplies": "Industrials",
    "Live Plant and Animal Material and Accessories and Supplies": "Consumer Staples",
    "Management and Business Professionals and Administrative Services": "Industrials",
    "Manufacturing Components and Supplies": "Industrials",
    "Material Handling and Conditioning and Storage Machinery and their Accessories and Supplies": "Industrials",
    "Medical Equipment and Accessories and Supplies": "Health Care",
    "Mineral and Textile and Inedible Plant and Animal Materials": "Materials",
    "Mining and oil and gas services": "Energy",
    "Mining and Well Drilling Machinery and Accessories": "Energy",
    "Musical Instruments and Games and Toys and Arts and Crafts and Educational Equipment and Materials and Accessories and Supplies": "Consumer Discretionary",
    "Office Equipment and Accessories and Supplies": "Industrials",
    "Organizations and Clubs": "Consumer Discretionary",
    "Paper Materials and Products": "Materials",
    "Personal and Domestic Services": "Consumer Discretionary",
    "Pharmaceutical Products incl. Contraceptives and Vaccines": "Health Care",
    "Politics and Civic Affairs Services": "Industrials",
    "Power Generation and Distribution Machinery and Accessories": "Industrials",
    "Printing and Photographic and Audio and Visual Equipment and Supplies": "Consumer Discretionary",
    "Public Order and Security and Safety Services": "Industrials",
    "Public Utilities and Public Sector Related Services": "Utilities",
    "Published Products": "Communication Services",
    "Resin and Rosin and Rubber and Foam and Film and Elastomeric Materials": "Materials",
    "Service Industry Machinery and Equipment and Supplies": "Industrials",
    "Sports and Recreational Equipment and Supplies and Accessories": "Consumer Discretionary",
    "Structures and Building and Construction and Manufacturing Components and Supplies": "Industrials",
    "Timepieces and Jewelry and Gemstone Products": "Consumer Discretionary",
    "Tools and General Machinery": "Industrials",
    "Transportation and Storage and Mail Services": "Industrials",
    "Travel and Food and Lodging and Entertainment Services": "Consumer Discretionary",
    "Vehicles and their Accessories and Components": "Consumer Discretionary",
}

sector_codes = {sector: i+10 for i, sector in enumerate(sorted(set(segment_to_sector.values())))}


In [4]:
unspsc['Sector Title'] = unspsc['Segment Title'].map(segment_to_sector)
unspsc['Sector'] = unspsc['Sector Title'].map(sector_codes)

In [5]:
def generate_code(row: pd.Series) -> str:
    """
    Generate a code based on the row's values.
    """
    if type(row['Commodity Title']) != str:
        if type(row['Class Title']) != str:
            if type(row['Family Title']) != str:
                if type(row['Segment Title']) != str:
                    return str(row['Sector'])+''
                else:
                    return str(row['Sector']) + str(row['Segment'])
            else:
                return str(row['Sector']) + str(row['Family'])
        else:
            return str(row['Sector']) + str(row['Class'])
    else:
        return str(row['Sector']) + str(row['Commodity'])

In [6]:
unspsc['Code'] = unspsc.apply(generate_code, axis=1)
unspsc['Code'] = unspsc['Code'].astype(float).astype(int)

In [7]:
unspsc = unspsc.drop(columns=['Sector', 'Segment', 'Family', 'Class', 'Commodity']).reindex(columns=['Code','Sector Title', 'Segment Title', 'Family Title', 'Class Title', 'Commodity Title'])

In [8]:
# Create new rows for each sector
new_rows = []
for sector, code in sector_codes.items():
    new_row = {
        'Code': int(str(code) + '0' * 8),
        'Sector Title': sector,
        'Segment Title': np.nan,
        'Family Title': np.nan,
        'Class Title': np.nan,
        'Commodity Title': np.nan
    }
    new_rows.append(new_row)

# Append new rows to the dataframe
unspsc = pd.concat([unspsc, pd.DataFrame(new_rows)], ignore_index=True)

In [9]:
"No duplicate codes" if len(unspsc['Code'].unique()) == unspsc.shape[0] else "Duplicate Codes found"

'No duplicate codes'

In [11]:
unspsc = unspsc.sort_values(by='Code').rename(columns={
    'Sector Title': 'tag1',
    'Segment Title': 'tag2',
    'Family Title': 'tag3',
    'Class Title': 'tag4',
    'Commodity Title': 'tag5'
})
unspsc

,Code,tag1,tag2,tag3,tag4,tag5
158463,1000000000,Communication Services,NaN,NaN,NaN,NaN
118063,1043000000,Communication Services,Information Technology Broadcasting and Teleco...,NaN,NaN,NaN
118475,1043190000,Communication Services,Information Technology Broadcasting and Teleco...,Communications Devices and Accessories,NaN,NaN
118438,1043191500,Communication Services,Information Technology Broadcasting and Teleco...,Communications Devices and Accessories,Personal communication devices,NaN
118418,1043191501,Communication Services,Information Technology Broadcasting and Teleco...,Communications Devices and Accessories,Personal communication devices,Mobile phones
...,...,...,...,...,...,...
154225,2083121705,Utilities,Public Utilities and Public Sector Related Ser...,Information services,Mass communication services,Social media related services
154231,2083130000,Utilities,Public Utilities and Public Sector Related Ser...,Information and communication broadcasting Ser...,NaN,NaN
154230,2083130100,Utilities,Public Utilities and Public Sector Related Ser...,Information and communication broadcasting Ser...,Communication service,NaN
154228,2083130101,Utilities,Public Utilities and Public Sector Related Ser...,Information and communication broadcasting Ser...,Communication service,Wire communication service


In [14]:
unspsc['code'] = unspsc['code'].astype(str).str.zfill(10).apply(lambda x: '.'.join([x[i:i+2] for i in range(0, 10, 2)]))

def adjust_code(row):
    if pd.isna(row['tag2']):
        return row['code'][:2]
    elif pd.isna(row['tag3']):
        return row['code'][:5]
    elif pd.isna(row['tag4']):
        return row['code'][:8]
    elif pd.isna(row['tag5']):
        return row['code'][:11]
    else:
        return row['code']

unspsc['code'] = unspsc.apply(adjust_code, axis=1)

In [15]:
unspsc

,code,tag1,tag2,tag3,tag4,tag5
0,10,Communication Services,NaN,NaN,NaN,NaN
1,10.43,Communication Services,Information Technology Broadcasting and Teleco...,NaN,NaN,NaN
2,10.43.19,Communication Services,Information Technology Broadcasting and Teleco...,Communications Devices and Accessories,NaN,NaN
3,10.43.19.15,Communication Services,Information Technology Broadcasting and Teleco...,Communications Devices and Accessories,Personal communication devices,NaN
4,10.43.19.15.01,Communication Services,Information Technology Broadcasting and Teleco...,Communications Devices and Accessories,Personal communication devices,Mobile phones
...,...,...,...,...,...,...
158469,20.83.12.17.05,Utilities,Public Utilities and Public Sector Related Ser...,Information services,Mass communication services,Social media related services
158470,20.83.13,Utilities,Public Utilities and Public Sector Related Ser...,Information and communication broadcasting Ser...,NaN,NaN
158471,20.83.13.01,Utilities,Public Utilities and Public Sector Related Ser...,Information and communication broadcasting Ser...,Communication service,NaN
158472,20.83.13.01.01,Utilities,Public Utilities and Public Sector Related Ser...,Information and communication broadcasting Ser...,Communication service,Wire communication service


In [16]:
unspsc.to_csv('./dados/tags/unspsc+.csv', index=False)